Model Deployment

Deploy the model with an API that others can use to train their own models and predict volatility. First,creating a model for all the code created in the previous notebooks. Then complete the main module, which will hold the FastAPI application with two paths: one for model training and one for prediction. 

In [40]:
%load_ext autoreload
%load_ext sql
%autoreload 2


import sqlite3
import requests
import os
from glob import glob

import pandas as pd
import matplotlib.pyplot as plt
from config import settings
import numpy as np
from arch import arch_model
from data import SQLRepository
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import joblib
from arch.univariate.base import ARCHModelResult
from data import SQLRepository
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [41]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

Model module

I already have code to build,train and make predictions using the Garch(1,1) model. To make the code reusable i'll put it in its own module.

First I'll instanciate a repository that will be used for testing the module as I build it

In [42]:
#Create a SQLRepository named repo. Attach it to a SQLite connection.
connection = sqlite3.connect(settings.db_name, check_same_thread=False)
repo = SQLRepository(connection=connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In the model module, create a definition for a GarchModel model class starting with an __init__ method then test the class using  assert statements

In [43]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_cic = GarchModel(ticker="CIC", repo=repo, use_new_data=False)

# Does `gm_cic` have the correct attributes?
assert gm_cic.ticker == "CIC"
assert gm_cic.repo == repo
assert not gm_cic.use_new_data
assert gm_cic.model_directory == settings.model_directory

Turn the wrangle_data function from the garch-models notebook into a method for the GarchModel class.Use assert statements to test the method by getting and wrangling data for  Co-operative bank

In [44]:
# Instantiate `GarchModel`, use new data
model_cic = GarchModel(ticker="CIC", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_cic, "data")

# Wrangle data
model_cic.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_cic, "data")

# Is the `data` a Series?
assert isinstance(model_cic.data, pd.Series)

# Is Series correct shape?
assert model_cic.data.shape == (1000,)

model_cic.data.head()

date
2019-03-22    0.197433
2019-03-25    0.000000
2019-03-26   -0.098522
2019-03-27    0.098619
2019-03-28    0.197044
Name: return, dtype: float64

Create a fit method for the GarchModel class. Test it.

In [45]:
# Instantiate `GarchModel`, use old data
model_cic = GarchModel(ticker="CIC", repo=repo, use_new_data=False)

# Wrangle data
model_cic.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_cic.fit(p=1, q=1)

# Does `model_cic` have a `model` attribute now?
assert hasattr(model_cic, "model")

# Is model correct data type?
assert isinstance(model_cic.model, ARCHModelResult)

# Does model have correct parameters?
assert model_cic.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_cic.model.summary()

/usr/local/python/3.10.4/lib/python3.10/site-packages/arch/univariate/base.py:753: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(


<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2705.76
Distribution:                  Normal   AIC:                           5419.52
Method:            Maximum Likelihood   BIC:                           5439.15
                                        No. Observations:                 1000
Date:                Sun, Mar 12 2023   Df Residuals:                      999
Time:                        08:27:27   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0237  6.073e-02      0.390      0.697 [-9.536e-02,  0.143]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.0357  7.471e-02      0.478      0.633   [ -0.111,  0.182]
alpha[1]       0.0882  3.102e-02      2.844  4.458e-03 [2.741e-02,  0.149]
beta[1]        0.9119  4.622e-02     19.730  1.191e-86   [  0.821,  1.002]
==========================================================================

Covariance estimator: robust
WARNING: The optimizer did not indicate successful convergence. The message was Positive directional derivative for linesearch.
See convergence_flag.

"""

Create a predict_volatility method for the GarchModel class. Your method will need to return predictions as a dictionary, so you'll need to add your clean_prediction function as a helper method. When you're done, test your work using the assert statements below.

In [46]:
# Generate prediction from `CIC`
prediction = model_cic.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

{'2023-03-13T00:00:00': 3.503484632245449,
 '2023-03-14T00:00:00': 3.5086987178190667,
 '2023-03-15T00:00:00': 3.5139054294333096,
 '2023-03-16T00:00:00': 3.5191047998440097,
 '2023-03-17T00:00:00': 3.524296861567689}

Saving a trained model to load it when need. I'll use the jolib library

Create a dump method for the GarchModel class to save the model assigned to the model attribute to the folder specified in the configuration settings. Test the method.

In [47]:
# Save `model_shop` model, assign filename
filename = model_cic.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_cic.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

'models/2023-03-12T08:27:27.582506_CIC.pkl'

Create a load function that will take a ticker symbol as input and return a model.

In [48]:
def load(ticker):

    """Load latest model from model directory.

    Parameters
    ----------
    ticker : str
        Ticker symbol for which model was trained.

    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = pattern = os.path.join(settings.model_directory, f"*{ticker}.pkl")


    # Try to find path of latest model
    try:
        model_path = model_path=sorted(glob(pattern))[-1]
   
    # Handle possible `IndexError`
    except IndexError:
        raise Exception(f"No model trained for '{ticker}'.")


    # Load model
    model = joblib.load(model_path)

    # Return model
    return model

Test the function

In [49]:
# Assign load output to `model`
model_cic = load(ticker="CIC")

# Does function return an `ARCHModelResult`
assert isinstance(model_cic, ARCHModelResult)

# Check model parameters
model_cic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2705.76
Distribution:                  Normal   AIC:                           5419.52
Method:            Maximum Likelihood   BIC:                           5439.15
                                        No. Observations:                 1000
Date:                Sun, Mar 12 2023   Df Residuals:                      999
Time:                        08:27:27   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0237  6.073e-02      0.390      0.697 [-9.536e-02,  0.143]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.0357  7.471e-02      0.478      0.633   [ -0.111,  0.182]
alpha[1]       0.0882  3.102e-02      2.844  4.458e-03 [2.741e-02,  0.149]
beta[1]        0.9119  4.622e-02     19.730  1.191e-86   [  0.821,  1.002]
==========================================================================

Covariance estimator: robust
WARNING: The optimizer did not indicate successful convergence. The message was Positive directional derivative for linesearch.
See convergence_flag.

"""

Transform the load function into a method for the GarchModel class.Test the method

In [50]:
model_cic = GarchModel(ticker="CIC", repo=repo, use_new_data=False)

# Check that new `model_cic` doesn't have model attached
assert not hasattr(model_cic, "model")

# Load model
model_cic.load()

# Does `model_cic` have model attached?
assert hasattr(model_cic, "model")

model_cic.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2705.76
Distribution:                  Normal   AIC:                           5419.52
Method:            Maximum Likelihood   BIC:                           5439.15
                                        No. Observations:                 1000
Date:                Sun, Mar 12 2023   Df Residuals:                      999
Time:                        08:27:27   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.0237  6.073e-02      0.390      0.697 [-9.536e-02,  0.143]
                             Volatility Model                             
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
omega          0.0357  7.471e-02      0.478      0.633   [ -0.111,  0.182]
alpha[1]       0.0882  3.102e-02      2.844  4.458e-03 [2.741e-02,  0.149]
beta[1]        0.9119  4.622e-02     19.730  1.191e-86   [  0.821,  1.002]
==========================================================================

Covariance estimator: robust
WARNING: The optimizer did not indicate successful convergence. The message was Positive directional derivative for linesearch.
See convergence_flag.

"""

Done with the model module, moving on to the main module that completes a FastApi interactive application for the model.

In the main module, instantiate a FastAPI application named app
Run it on a server. In this case, we'll run the server on our virtual machine using the uvicorn library

Run the command uvicorn main:app --reload --workers 1 --host localhost --port 8008 to start up the app server

In [51]:
#Create a "/hello" path for your app that returns a greeting when it receives a get request.
#Create a get request to hit the "/hello" path running at "http://localhost:8008".

url = "http://localhost:8008/hello"
response = requests.get(url=url)

print("response code:", response.status_code)
response.json()

response code: 200


{'message': 'Hello world!'}

/fit" Path

The first path will allow the user to fit a model to stock data when they make a post request to the server. They'll have the choice to use new data from AlphaVantage, or older data that's already in the database. When a user makes a request, they'll receive a response telling them if the operation was successful or whether there was an error.

The FastApi will use the pydantic library, which checks that each request has the correct parameters and data types. It does this by using special data classes I'll define in the "/fit" path classes. The "/fit" path will take user input and then output a response,these two will be my classes

Create definitions for a FitIn and a FitOut data class. The FitIn class inherits from the pydantic BaseClass, and the FitOut class inherits from the FitIn class. I'll include type hints.

Test whether pydantic ensures that users are supplying the correct input and the application is returning the correct output

In [52]:
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker="CIC",
    use_new_data= True,
    n_observations= 2000,
    p=1,
    q=1
)

print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = FitOut(
    ticker="CIC",
    use_new_data= True,
    n_observations= 2000,
    p=1,
    q=1,
    success= True,
    message="Method is ready to rock!"
)
print(fo)

ticker='CIC' use_new_data=True n_observations=2000 p=1 q=1
ticker='CIC' use_new_data=True n_observations=2000 p=1 q=1 success=True message='Method is ready to rock!'


Next, i'll create a function that instantiates a GarchModel object each time a user makes a request

In [53]:
#Create a build_model function in the main module.
from main import build_model

# Instantiate `GarchModel` with function
model_coop = build_model(ticker="COOP", use_new_data=False)

# Is `SQLRepository` attached to `model_coop`?
assert isinstance(model_coop.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_coop.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_coop.ticker == "COOP"

# Is `use_new_data` attribute correct?
assert not model_coop.use_new_data

model_coop

I now have a data classes,a build_model function. So I'll just build the "/fit" path and include features, like error handling.


So I'll create a "/fit" path for the app. It will take a FitIn object as input, and then build a GarchModel using the build_model function. The model will wrangle the needed data, fit to the data, and save the completed model. Finally, it will send a response in the form of a FitOut object. It will also handle any errors that may arise.

In [54]:
#Create a post request to hit the "/fit" path running at "http://localhost:8008". 
#Train a GARCH(1,1) model on 2000 observations of Equity data that's already downloaded. Pass in the parameters as a dictionary using the json argument.

# URL of `/fit` path
url = "http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker" : "EQTY",
    "use_new_data": True,
    "n_observations": 2000,
    "p":1,
    "q":1
    
}
# Response of post request
response = requests.post(url=url, json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

response code: 200


{'ticker': 'EQTY',
 'use_new_data': True,
 'n_observations': 2000,
 'p': 1,
 'q': 1,
 'success': True,
 'message': "trained and saved 'models/2023-03-12T08:27:28.437621_EQTY.pkl'.Metrics: AIC 205.3144, BIC 213.6918."}

"/predict" Path

For the `"/predict"` path, users will be able to make a `post` request with the ticker symbol they want a prediction for and the number of days they want to forecast into the future. The app will return a forecast or, if there's an error, a message explaining the problem.

I'll start with data classes for the in- and output.

So I'll Create definitions for a PredictIn and PredictOut data class. The PredictIn class should inherit from the pydantic BaseModel, and the PredictOut class should inherit from the PredictIn class.I will also include type hints then test the classes.

In [55]:
from main import PredictIn, PredictOut

pi = PredictIn(ticker="CIC", n_days=5)
print(pi)

po = PredictOut(
    ticker="CIC", n_days=5, success=True, forecast={}, message="success"
)
print(po)

ticker='CIC' n_days=5
ticker='CIC' n_days=5 success=True forecast={} message='success'


Next i'll create a "/predict" path for the app. It will take a PredictIn object as input, build a GarchModel, load the most recent trained model for the given ticker, and generate a dictionary of predictions. It will then return a PredictOut object with the predictions included and handle any errors that may arise.

Next I'll create a post request to hit the "/predict" path running at "http://localhost:8008". 

In [56]:
# URL of `/predict` path
url = "http://localhost:8008/predict"
# Data to send to path
json = {"ticker": "CIC","n_days":3}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

{'ticker': 'CIC',
 'n_days': 3,
 'success': True,
 'forecast': {'2023-03-13T00:00:00': 3.503484632245449,
  '2023-03-14T00:00:00': 3.5086987178190667,
  '2023-03-15T00:00:00': 3.5139054294333096},
 'message': 'Done'}